In [1]:
import pandas as pd
from pricer.pricer import Pricer
from pricer.pricer_factory import PricerFactory
from database.market import Market
from database.adatabase import ADatabase
from processor.processor import Processor as p
from datetime import datetime, timedelta
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler
import pickle

In [2]:
## initializing pricer_class
pricer_class = PricerFactory.build(Pricer.DAILY_STOCK_SPECULATION)

In [3]:
backtest_start_date = datetime(2001,1,1)
backtest_end_date = datetime(2023,1,1)
current_year = datetime.now().year

In [4]:
## creating database objects
market = Market()
modeler_strat = UniversalModeler()

In [5]:
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()
sp500 = sp500.rename(columns={"Symbol":"ticker"})

## Historical Training and Prediction Set

In [6]:
market.connect()
training_sets = []
for ticker in tqdm(sp500["ticker"].unique()):
    try:
        prices = market.retrieve_ticker_prices(pricer_class.asset_class.value,ticker)
        prices = p.column_date_processing(prices)
        ticker_data = pricer_class.training_set(ticker,prices,False)
        training_sets.append(ticker_data)
    except Exception as e:
        print(str(e))
        continue
market.disconnect()
data = pd.concat(training_sets)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [00:40<00:00, 12.01it/s]


In [7]:
training_data = data.dropna().copy().sort_values(["year",pricer_class.time_horizon_class.naming_convention])

In [8]:
training_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,year,date,ticker,adjclose,y
13,25.663534,25.714893,26.709646,26.255519,26.844802,27.247569,27.247569,27.166475,27.653039,27.788195,27.247569,25.747331,24.498484,25.512158,2000,2000-01-21,MMM,25.663534,25.071548
13,2.451236,2.368832,2.410583,2.368832,2.403991,2.355648,2.321587,2.403991,2.430360,2.403991,2.307304,2.341364,2.349055,2.383116,2000,2000-01-21,AOS,2.451236,2.472112
13,7.812441,8.027853,8.280837,8.453667,8.641527,8.453667,8.531316,8.684689,8.809648,8.872128,8.779658,8.449765,8.497249,8.747169,2000,2000-01-21,ABT,7.812441,7.514371
13,29.440000,23.500000,24.375000,26.000000,21.375000,21.000000,19.595000,19.815000,20.500000,17.940000,18.030000,18.065000,17.815000,18.250000,2000,2000-01-21,ABMD,29.440000,31.375000
13,1.117958,1.168203,1.237660,1.256132,1.251699,1.264999,1.251699,1.307116,1.251699,1.191109,1.159336,1.182242,1.177809,1.214754,2000,2000-01-21,ATVI,1.117958,1.108352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5924,137.540000,135.880000,135.930000,133.840000,135.740000,135.560000,135.950000,135.100000,134.170000,134.810000,133.450000,133.900000,136.660000,137.270000,2023,2023-07-21,YUM,137.540000,137.940000
5924,306.460000,304.540000,318.370000,313.170000,309.510000,309.280000,318.470000,315.930000,309.160000,306.160000,295.220000,291.140000,292.850000,296.270000,2023,2023-07-21,ZBRA,306.460000,302.590000
5531,142.230000,142.730000,140.790000,140.860000,140.660000,142.370000,143.110000,142.650000,143.180000,142.510000,140.860000,141.200000,142.360000,142.510000,2023,2023-07-21,ZBH,142.230000,140.870000
5924,36.110000,37.900000,34.460000,32.430000,31.360000,30.700000,32.000000,31.280000,30.425000,29.560000,28.980000,27.780000,27.960000,28.140000,2023,2023-07-21,ZION,36.110000,37.050000


## Historical Modeling and Simulation Code

In [9]:
# historical model and simulation
pricer_class.db.connect()
model_start_date = backtest_start_date - timedelta(pricer_class.time_horizon_class.model_date_offset)
model_end_date = backtest_end_date - timedelta(pricer_class.time_horizon_class.model_date_offset)
for year in range(model_start_date.year,model_end_date.year):
    training_slice = training_data[(training_data["year"]<year) & (training_data["year"] >= year - 4)].reset_index(drop=True)
    prediction_set = training_data[training_data["year"]==year].reset_index(drop=True)
    stuff = modeler_strat.model(training_slice,prediction_set,pricer_class.factors,False)
    stuff = stuff.rename(columns={"prediction":f"price_prediction"})
    relevant_columns = list(set(list(stuff.columns)) - set(pricer_class.factors))
    stuff["date"] = [x + timedelta(days=1) if x.weekday() < 4 else x + timedelta(days=2) for x in stuff["date"]]
    pricer_class.db.store("sim",stuff[relevant_columns])
pricer_class.db.disconnect()

## Current Year Modeling Code

In [10]:
pricer_class.db.connect()
training_slice = training_data[(training_data["year"]<current_year) & (training_data["year"] >= current_year - 4)].reset_index(drop=True)
prediction_set = training_data[training_data["year"]==current_year].reset_index(drop=True)
stuff = modeler_strat.recommend_model(training_slice,pricer_class.factors,False)
stuff["model"] = [pickle.dumps(x) for x in stuff["model"]]
stuff["year"] = current_year
pricer_class.db.store("models",stuff)
pricer_class.db.disconnect()
pricer_class.cloud_connect()
pricer_class.store("models",stuff)
pricer_class.disconnect()

AttributeError: 'Speculation' object has no attribute 'cloud_connect'